---
### **Objetivo**

Realizar o reconhecimento inicial das bases de dados, validar sua qualidade e integridade, e executar tratamentos de limpeza para preparar os dados para as etapas subsequentes de análise

---

### **Carregamento dos dados e importação de bibliotecas**

In [1]:
# Bibliotecas
import pandas as pd
from ydata_profiling import ProfileReport
import os

# Configurações Tabela
pd.set_option("display.max_columns", None)

In [2]:
# Carregamento dos dados
df_transactions = pd.read_excel('data/raw/aml_case.xlsx', sheet_name='Transactions')
df_kyc_profiles = pd.read_excel('data/raw/aml_case.xlsx', sheet_name='KYC_Profiles')
df_merchants = pd.read_excel('data/raw/aml_case.xlsx', sheet_name='Merchants')
df_geo_behavior = pd.read_excel('data/raw/aml_case.xlsx', sheet_name='GeoBehavior')

### **Reconhecimento e Limpeza dos Datasets**

In [3]:
# Funções EDA
def eda(dataframe, title, output_dir="eda_reports"):    
    
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{title}.html")
    
    profile = ProfileReport(dataframe, title=title, explorative=True)
    profile.to_file(output_path)
    
def shape_info(dataframe, name="DataFrame"):
    n_cols = dataframe.shape[1]
    n_rows = dataframe.shape[0]
    print(f"{name} - DataFrame has {n_rows} rows and {n_cols} columns.")

def unique_values(dataframe, id_columns=None):
    unique_counts = dataframe.nunique()
    total_counts = dataframe.count()
    duplicate_counts = total_counts - unique_counts
    
    result = pd.DataFrame({
        'Unique Values': unique_counts,
        'Total Values': total_counts,
        'Duplicates': duplicate_counts
    })
    
    if id_columns:
        for col in id_columns:
            if col in dataframe.columns:
                dups = dataframe[col].duplicated().sum()
                print(f"{col}: {dups} duplicates")
    
    return result

def missing_values(dataframe):
    missing = dataframe.isnull().sum()
    missing = missing[missing > 0]
    missing_percent = (missing / len(dataframe)) * 100
    return pd.DataFrame({'Missing Values': missing, 'Percentage': missing_percent})

# Funções de tratamento
def convert_data_types(dataframe, mapping, source_name):

    dataframe = dataframe.copy()
    
    for column, dtype in mapping.items():
        if column not in dataframe.columns:
            print(f"[{source_name}] Column '{column}' not found in DataFrame")
            continue
        
        try:
            original_dtype = dataframe[column].dtype
            
            if dtype == 'datetime64[ns]':
                dataframe[column] = pd.to_datetime(dataframe[column], errors='coerce')
            
            elif dtype == 'boolean':
                # Maps Yes/No to True/False, preserving NaN/None
                dataframe[column] = dataframe[column].map({'Yes': True, 'No': False}).astype('boolean')
            
            elif isinstance(dtype, pd.CategoricalDtype):
                # Converts to ordered categorical
                dataframe[column] = pd.Categorical(dataframe[column], categories=dtype.categories, ordered=dtype.ordered)
            
            elif dtype == 'object':
                # Ensures text fields preserve leading zeros
                dataframe[column] = dataframe[column].astype(str).replace('nan', pd.NA).astype('object')
            
            else:
                dataframe[column] = dataframe[column].astype(dtype)
            
            print(f"[{source_name}] '{column}': {original_dtype} → {dataframe[column].dtype}")
        
        except Exception as e:
            print(f"[{source_name}] Error converting '{column}': {e}")
    
    return dataframe

### **1. Volume de dados**

In [4]:
# Volume de dados dos dataframes
shape_info(df_transactions, "Transactions")
shape_info(df_kyc_profiles, "KYC Profiles") 
shape_info(df_merchants, "Merchants")
shape_info(df_geo_behavior, "Geo Behavior")    

Transactions - DataFrame has 52000 rows and 41 columns.
KYC Profiles - DataFrame has 2500 rows and 16 columns.
Merchants - DataFrame has 1000 rows and 10 columns.
Geo Behavior - DataFrame has 3497 rows and 6 columns.


### **2. Overview**

#### **2.1. Transactions Overview**

In [5]:
# Verificando as primeiras entradas
display(df_transactions.head(3))

# Relatório automático de EDA
profile_report_transactions = eda(df_transactions, "Transactions Data EDA")

,transaction_id,timestamp,transaction_type,sender_id,sender_entity_type,receiver_id,receiver_entity_type,amount_brl,amount_orig,currency,fx_to_brl,status,channel,capture_method,payment_method,installments,issuing_or_acquiring,pix,pix_flow,card_brand,card_present,auth_3ds,eci,mcc,geo_country,geolocation_lat,geolocation_lon,ip_country,ip_anomaly,ip_proxy_vpn_tor,device_fingerprint,ip_address,device_rooted,sender_country,receiver_country,country_risk_geo,country_risk_ip,country_risk_sender,country_risk_receiver,cross_border,sanctions_screening_hit
0,T9HIMVHJ8TMK7,2025-09-29T01:19:37,PIX,C100602,customer,M200223,merchant,3478.73,3478.73,BRL,1,Confirmed,App,CopyPaste,NaN,1,NaN,Yes,cash_out,NaN,NaN,NaN,NaN,6051,BR,-11.743364,-54.286939,DE,No,NaN,poq50ptmz8g3pgoh,193.59.218.134,No,BR,BR,Low,Low,Low,Low,No,No
1,TRY1GG0393ZEQ,2025-09-07T00:14:11,PIX,C101190,customer,M200471,merchant,3356.15,3356.15,BRL,1,Confirmed,App,Pix Key,NaN,1,NaN,Yes,cash_out,NaN,NaN,NaN,NaN,5945,BR,-0.031356,-62.010354,BR,No,NaN,ve5ar34n3q5arhcn,118.247.245.154,No,BR,BR,Low,Low,Low,Low,No,No
2,TYVX10N3OXT1H,2025-07-01T22:36:33,PIX,C101811,customer,C101070,customer,1595.14,1595.14,BRL,1,Confirmed,App,CopyPaste,NaN,1,NaN,Yes,cash_out,NaN,NaN,NaN,NaN,4111,BR,-17.208767,-66.365148,BR,No,NaN,bnv7ad7zrmn7cgfh,169.185.50.15,No,BR,BR,Low,Low,Low,Low,No,No


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:01<00:00, 23.58it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

`Transactions` contém registros de operações financeiras com informações de pagamento, geolocalização, dispositivos, sinais de risco, entre outras. Os dados abrangem o período de 3 meses aproximadamente (01/07/2025 até 04/10/2025)

**Dicionário de dados**
| # | Variável | Descrição da Variável | Exemplos  |
|---|----------|----------------------|---------------------------|
| 1 | transaction_id | ID único e exclusivo de cada transação. | T9HIMVHJ8TMK7, TRY1GG0393ZEQ |
| 2 | timestamp | Data e hora em que a transação foi iniciada. | 2025-09-29T01:19:37 |
| 3 | transaction_type | Meio de pagamento utilizado (PIX, Card ou Wire). | PIX, Card, Wire |
| 4 | sender_id | Identificador único da conta/entidade que envia o valor. | C100602, C101190, M200612 |
| 5 | sender_entity_type | Indica se o remetente é cliente final ou estabelecimento. | customer, merchant |
| 6 | receiver_id | Identificador único da conta/entidade que recebe o valor. | M200223, M200471, C101070 |
| 7 | receiver_entity_type | Indica se o recebedor é estabelecimento ou cliente final. | merchant, customer |
| 8 | amount_brl | Valor total da transação em Reais (BRL). | 3478.73, 3356.15, 1595.14 |
| 9 | amount_orig | Valor na moeda original da transação. | 3478.73, 3356.15, 500.00 |
| 10 | currency | Código da moeda de origem (BRL, EUR, USD). | BRL, EUR, USD |
| 11 | fx_to_brl | Taxa de câmbio utilizada para a conversão em BRL. | 1, 6, 5 |
| 12 | status | Estado atual da transação (Confirmed, Pending, Failed, etc.). | Confirmed, Pending, Failed |
| 13 | channel | Canal de origem da transação (App, API, Web, Terminal). | App, API, Web, Terminal |
| 14 | capture_method | Método técnico de captura (Pix Key, QR Code, Magstripe, etc.). | CopyPaste, Pix Key, QR Static |
| 15 | payment_method | Modalidade de pagamento (Crédito ou Débito). | credit, debit |
| 16 | installments | Número de parcelas. | 1, 10, 6 |
| 17 | issuing_or_acquiring | Papel da instituição (Emissora ou Adquirente). | acquiring, issuing |
| 18 | pix | Indica se a transação utiliza o trilho PIX. | Yes, No |
| 19 | pix_flow | Direção do fluxo PIX (cash_in ou cash_out para o cliente). | cash_out, cash_inc |
| 20 | card_brand | Bandeira do cartão (Visa, Mastercard, Elo, etc.). | Visa, Elo, Mastercard |
| 21 | card_present | Indica se o cartão estava fisicamente presente na transação. | Yes, No |
| 22 | auth_3ds | Indica se houve autenticação 3-D Secure (e-commerce). | No, Yes |
| 23 | eci | Indicador de condição do comércio eletrônico. | 7.0, 6.0, 5.0 |
| 24 | mcc | Código de Categoria do Merchant (ramo de atuação). | 6051, 5945, 4111 |
| 25 | geo_country | País baseado nas coordenadas de GPS/Geolocalização. | BR, SY, AE |
| 26 | geolocation_lat | Latitude registrada no momento da operação. | -11.743364, -0.031356 |
| 27 | geolocation_lon | Longitude registrada no momento da operação. | -54.286939, -62.010354 |
| 28 | ip_country | País identificado através do endereço IP do dispositivo. | DE, BR, RU |
| 29 | ip_anomaly | Indica anomalia técnica detectada na conexão IP. | No, Yes |
| 30 | ip_proxy_vpn_tor | Identifica uso de ferramentas de ocultação (VPN, Proxy, Tor). | VPN, Proxy, Tor |
| 31 | device_fingerprint | Assinatura digital única do hardware utilizado. | poq50ptmz8g3pgoh |
| 32 | ip_address | Endereço IP público do originador da transação. | 193.59.218.134, 118.247.245.154 |
| 33 | device_rooted | Indica se o dispositivo possui acesso Root ou Jailbreak. | No, Yes |
| 34 | sender_country | País de origem da conta do remetente. | BR, US, KP |
| 35 | receiver_country | País de origem da conta do recebedor. | BR, SY, AE |
| 36 | country_risk_geo | Nível de risco atribuído ao país da geolocalização. | Low, High, Monitored |
| 37 | country_risk_ip | Nível de risco atribuído ao país identificado pelo IP. | Low, Monitored, High |
| 38 | country_risk_sender | Nível de risco atribuído ao país do remetente. | Low, High, Monitored |
| 39 | country_risk_receiver | Nível de risco atribuído ao país do recebedor. | Low, High, Monitored |
| 40 | cross_border | Indica se a transação envolve países diferentes. | No, Yes |
| 41 | sanctions_screening_hit | Alerta de presença em listas de sanções (OFAC, ONU, etc). | No, Yes |


#### **2.2. KYC Profiles Overview**

In [6]:
# Verificando as primeiras entradas
display(df_kyc_profiles.head(3))

# Relatório automático de EDA
profile_report_kyc_profiles = eda(df_kyc_profiles, "KYC Data EDA")

,customer_id,full_name,cpf_cnpj,date_of_birth,annual_income_brl,declared_occupation,risk_rating,registration_date,country,state,city,beneficial_owner,pep,kyc_tier,kyc_risk_score,sanctions_list_hit
0,C100000,Customer 1,52601815908,1977-04-06,46542,Dentist,Low,2024-09-08,BR,SP,São Paulo,No,No,L1,50,No
1,C100001,Customer 2,18609139099,1993-08-03,10005,Student,High,2020-07-09,BR,SP,São Paulo,No,No,L3,60,No
2,C100002,Customer 3,46281948219,2009-03-31,30667,Freelancer,Low,2021-02-03,BR,AM,Manaus,Yes,No,L3,72,No


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:00<00:00, 437.90it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

`KYC_Profiles` contém os perfis cadastrais e regulatórios de clientes (PEP, sanções, classificação de risco)

**Dicionário de dados**
| # | Variável | Descrição da Variável | Exemplos |
|---|----------|----------------------|----------|
| 1 | customer_id | Identificador único do cliente. | C100000, C100001, C100002 |
| 2 | full_name | Nome completo do cliente cadastrado. | Customer 1, Customer 2, Customer 3 |
| 3 | cpf_cnpj | Documento de identificação (CPF ou CNPJ). | 52601815908, 18609139099 |
| 4 | date_of_birth | Data de nascimento (utilizada para cálculo de idade). | 1977-04-06, 1993-08-03 |
| 5 | annual_income_brl | Renda anual declarada pelo cliente em Reais. | 46542, 10005, 30667 |
| 6 | declared_occupation | Ocupação ou profissão declarada. | Dentist, Student, Freelancer |
| 7 | risk_rating | Classificação de risco atribuída no onboarding. | Low, High, Medium |
| 8 | registration_date | Data em que o cliente se cadastrou na fintech. | 2024-09-08, 2020-07-09 |
| 9 | country | País de residência do cliente. | BR |
| 10 | state | Estado (UF) de residência. | SP, AM, PR |
| 11 | city | Cidade de residência. | São Paulo, Manaus, Curitiba |
| 12 | beneficial_owner | Indica se o cliente é o próprio beneficiário final. | No, Yes |
| 13 | pep | Indica se o cliente é Pessoa Exposta Politicamente. | No, Yes |
| 14 | kyc_tier | Nível de profundidade do processo de KYC. | L1, L3, L2 |
| 15 | kyc_risk_score | Pontuação numérica detalhada de risco. | 50, 60, 72 |
| 16 | sanctions_list_hit | Presença em listas de sanções globais. | No, Yes |


#### **2.3. Merchants Overview**

In [7]:
# Verificando as primeiras entradas
display(df_merchants.head(3))

# Relatório automático de EDA
profile_report_merchants = eda(df_merchants, "Merchants Data EDA")

,merchant_id,merchant_name,cnpj,mcc,country,city,owner_customer_id,merchant_high_risk_flag,merchant_chargeback_ratio_90d,mcc_risk
0,M200000,Merchant 1,65240421527930,5411,BR,NaN,C101280,No,0.034,Normal
1,M200001,Merchant 2,96297252396610,7995,BR,NaN,NaN,No,0.052,High
2,M200002,Merchant 3,67551536706595,5411,BR,NaN,C100993,No,0.060,Normal


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 525.39it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

`Merchants` contém os cadastros de estabelecimentos comerciais com métricas de risco e vínculos proprietários.

**Dicionário de dados**
| # | Variável | Descrição da Variável | Exemplos |
|---|----------|----------------------|----------|
| 1 | merchant_id | Identificador único do estabelecimento. | M200000, M200001, M200002 |
| 2 | merchant_name | Nome fantasia do estabelecimento comercial. | Merchant 1, Merchant 2, Merchant 3 |
| 3 | cnpj | Cadastro Nacional da Pessoa Jurídica (CNPJ). | 65240421527930, 96297252396610 |
| 4 | mcc | Merchant Category Code (Ramo de Atividade). | 5411, 7995, 5812 |
| 5 | country | País de registro do estabelecimento. | BR, GB, CN |
| 6 | city | Cidade de registro do merchant. | São Paulo, Curitiba (ou n/a) |
| 7 | owner_customer_id | ID do cliente (PF) proprietário do merchant. | C101280, C100993, C101098 |
| 8 | merchant_high_risk_flag | Indicador de alto risco baseado no setor/país. | No, Yes |
| 9 | merchant_chargeback_ratio_90d | Taxa de chargebacks nos últimos 90 dias. | 0.034, 0.052, 0.06 |
| 10 | mcc_risk | Nível de risco intrínseco à categoria de negócio. | Normal, High |

#### **2.4. GeoBehavior Overview**

In [8]:
# Verificando as primeiras entradas
display(df_geo_behavior.head(3))

# Relatório automático de EDA
profile_report_geo_behavior = eda(df_geo_behavior, "Geo Behavior Data EDA")

,sender_id,tx_count,counterparties,avg_amount,tx_window_days,tx_per_day
0,C100000,18,18,3517.62,83,0.217
1,C100001,16,16,3789.51,83,0.193
2,C100002,25,25,3895.57,93,0.269


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 39.08it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

`GeoBehavior` contém padrões comportamentais agregados por remetente.

**Dicionário de dados**
| # | Variável | Descrição da Variável | Exemplos |
|---|----------|----------------------|----------|
| 1 | sender_id | Identificador do remetente analisado. | C100000, C100001, C100002 |
| 2 | tx_count | Volume total de transações enviadas no período. | 18, 16, 25 |
| 3 | counterparties | Número de beneficiários distintos atendidos. | 18, 16, 25 |
| 4 | avg_amount | Valor médio (Ticket Médio) das transações. | 3517.62, 3789.51, 3895.57 |
| 5 | tx_window_days | Total de dias de atividade observada. | 83, 93, 87 |
| 6 | tx_per_day | Frequência diária média de transações. | 0.217, 0.193, 0.269 |

### **3. Tipo de dados das variáveis**

In [9]:
#Verificando os tipos de dados Transactions
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52000 entries, 0 to 51999
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           52000 non-null  object 
 1   timestamp                52000 non-null  object 
 2   transaction_type         52000 non-null  object 
 3   sender_id                52000 non-null  object 
 4   sender_entity_type       52000 non-null  object 
 5   receiver_id              52000 non-null  object 
 6   receiver_entity_type     52000 non-null  object 
 7   amount_brl               52000 non-null  float64
 8   amount_orig              52000 non-null  float64
 9   currency                 52000 non-null  object 
 10  fx_to_brl                52000 non-null  int64  
 11  status                   52000 non-null  object 
 12  channel                  52000 non-null  object 
 13  capture_method           52000 non-null  object 
 14  payment_method        

In [10]:
#Verificando os tipos de dados KYC Profiles
df_kyc_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   customer_id          2500 non-null   object
 1   full_name            2500 non-null   object
 2   cpf_cnpj             2500 non-null   int64 
 3   date_of_birth        2500 non-null   object
 4   annual_income_brl    2500 non-null   int64 
 5   declared_occupation  2500 non-null   object
 6   risk_rating          2500 non-null   object
 7   registration_date    2500 non-null   object
 8   country              2500 non-null   object
 9   state                2500 non-null   object
 10  city                 2500 non-null   object
 11  beneficial_owner     2500 non-null   object
 12  pep                  2500 non-null   object
 13  kyc_tier             2500 non-null   object
 14  kyc_risk_score       2500 non-null   int64 
 15  sanctions_list_hit   2500 non-null   object
dtypes: int

In [11]:
#Verificando os tipos de dados Merchants
df_merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   merchant_id                    1000 non-null   object 
 1   merchant_name                  1000 non-null   object 
 2   cnpj                           1000 non-null   int64  
 3   mcc                            1000 non-null   int64  
 4   country                        1000 non-null   object 
 5   city                           0 non-null      float64
 6   owner_customer_id              349 non-null    object 
 7   merchant_high_risk_flag        1000 non-null   object 
 8   merchant_chargeback_ratio_90d  1000 non-null   float64
 9   mcc_risk                       1000 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 78.3+ KB


In [12]:
#Verificando os tipos de dados Geo Behavior
df_geo_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3497 entries, 0 to 3496
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sender_id       3497 non-null   object 
 1   tx_count        3497 non-null   int64  
 2   counterparties  3497 non-null   int64  
 3   avg_amount      3497 non-null   float64
 4   tx_window_days  3497 non-null   int64  
 5   tx_per_day      3497 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 164.1+ KB


#### **3.1. Correção de tipos de dados**
As tabelas `transactions`, `kyc_profiles` e `merchants` possuem alguns tipos de dados incoerentes com a informação armazenada, sendo necessário realizar esse tratamento. As correções incluem conversão de datas, booleanos nullable e variáveis de risco como categorias ordenadas.

In [13]:
# Mappings de correção de tipos
DTYPE_MAPPINGS = {
    'df_transactions': {
        'timestamp': 'datetime64[ns]',
        'fx_to_brl': 'float64',
        'pix': 'boolean',
        'card_present': 'boolean',
        'auth_3ds': 'boolean',
        'ip_anomaly': 'boolean',
        'device_rooted': 'boolean',
        'cross_border': 'boolean',
        'sanctions_screening_hit': 'boolean',
        'country_risk_geo': pd.CategoricalDtype(categories=['Low', 'Monitored', 'High'], ordered=True),
        'country_risk_ip': pd.CategoricalDtype(categories=['Low', 'Monitored', 'High'], ordered=True),
        'country_risk_sender': pd.CategoricalDtype(categories=['Low', 'Monitored', 'High'], ordered=True),
        'country_risk_receiver': pd.CategoricalDtype(categories=['Low', 'Monitored', 'High'], ordered=True)
    },
    'df_kyc_profiles': {
        'date_of_birth': 'datetime64[ns]',
        'registration_date': 'datetime64[ns]',
        'beneficial_owner': 'boolean',
        'pep': 'boolean',
        'sanctions_list_hit': 'boolean',
        'risk_rating': pd.CategoricalDtype(categories=['Low', 'Medium', 'High'], ordered=True)
    },
    'df_merchants': {
        'city': 'object',
        'merchant_high_risk_flag': 'boolean',
        'mcc_risk': pd.CategoricalDtype(categories=['Normal', 'High'], ordered=True)
    }
}

In [14]:
# Correção dos tipos de dados
df_transactions = convert_data_types(df_transactions, DTYPE_MAPPINGS['df_transactions'], 'df_transactions')

df_kyc_profiles = convert_data_types(df_kyc_profiles, DTYPE_MAPPINGS['df_kyc_profiles'],'df_kyc_profiles')

df_merchants = convert_data_types(df_merchants, DTYPE_MAPPINGS['df_merchants'], 'df_merchants')

[df_transactions] 'timestamp': object → datetime64[ns]
[df_transactions] 'fx_to_brl': int64 → float64
[df_transactions] 'pix': object → boolean
[df_transactions] 'card_present': object → boolean
[df_transactions] 'auth_3ds': object → boolean
[df_transactions] 'ip_anomaly': object → boolean
[df_transactions] 'device_rooted': object → boolean
[df_transactions] 'cross_border': object → boolean
[df_transactions] 'sanctions_screening_hit': object → boolean
[df_transactions] 'country_risk_geo': object → category
[df_transactions] 'country_risk_ip': object → category
[df_transactions] 'country_risk_sender': object → category
[df_transactions] 'country_risk_receiver': object → category
[df_kyc_profiles] 'date_of_birth': object → datetime64[ns]
[df_kyc_profiles] 'registration_date': object → datetime64[ns]
[df_kyc_profiles] 'beneficial_owner': object → boolean
[df_kyc_profiles] 'pep': object → boolean
[df_kyc_profiles] 'sanctions_list_hit': object → boolean
[df_kyc_profiles] 'risk_rating': obje

### **4. Verificação de valores ausentes**


As tabelas `transactions` e `merchants` apresentam valores ausentes que requerem investigação para validar a qualidade dos dados.

#### **4.1. Verificação de valores ausentes - Transactions**

Com base na análise de nulidade de Transactions, identificamos que os valores ausentes seguem padrões estruturais relacionados ao tipo de transação, não representando dados faltantes ou problemas de qualidade.

**Nulos estruturais por método de pagamento:**
   - Campos de cartão (`card_brand`, `card_present`, `issuing_or_acquiring`) são naturalmente nulos para PIX/Wire (~65,5%)
   - Campo `pix_flow` é naturalmente nulo para Card/Wire (~39,3%)
   - Esses nulos carregam informação: indicam o tipo de transação

**Nulos estruturais por método de captura:**
   - `auth_3ds` e `eci` são nulos para métodos que não suportam autenticação 3DS (~86,4%)
   - Aplicável apenas para E-commerce e MOTO

**Campo de segurança com alta nulidade:**
   - `ip_proxy_vpn_tor` (92,96% nulos) indica ausência de detecção de anonimização
   - Quando preenchido,  pode representar sinal de alerta importante para análise de risco
   - Imputar ou remover eliminaria esse indicador crítico

**Conclusão:** Os valores nulos são informativos e refletem a natureza heterogênea das transações. Mantê-los preserva a integridade dos dados e permite análises condicionais por tipo de transação/captura.

In [15]:
missing_values(df_transactions)

,Missing Values,Percentage
payment_method,34070,65.519231
issuing_or_acquiring,34070,65.519231
pix_flow,20453,39.332692
card_brand,34070,65.519231
card_present,34070,65.519231
auth_3ds,44905,86.355769
eci,44905,86.355769
ip_proxy_vpn_tor,48337,92.955769


In [16]:
count_transaction_type = df_transactions['transaction_type'].value_counts() / df_transactions.shape[0]
count_transaction_type

transaction_type
PIX     0.606673
Card    0.344808
Wire    0.048519
Name: count, dtype: float64

In [17]:
count_capture_method  = df_transactions['capture_method'].value_counts() / df_transactions.shape[0]
count_capture_method 

capture_method
QR Dynamic    0.153000
CopyPaste     0.152808
QR Static     0.151135
Pix Key       0.149731
Magstripe     0.070288
Chip          0.069462
MOTO          0.068827
NFC           0.068615
E-commerce    0.067615
SWIFT         0.024442
Domestic      0.024077
Name: count, dtype: float64

In [18]:
count_ip_proxy_vpn_tor  = df_transactions['ip_proxy_vpn_tor'].value_counts()
count_ip_proxy_vpn_tor 

ip_proxy_vpn_tor
VPN      2124
Proxy     996
Tor       543
Name: count, dtype: int64

#### **4.2. Verificação de valores ausentes - Merchants**

In [19]:
missing_values(df_merchants)

,Missing Values,Percentage
city,1000,100.0
owner_customer_id,651,65.1


In [20]:
df_merchants = df_merchants.drop('city', axis=1)

### **5. Verificação de registros com dados duplicados**

Com base na análise de duplicatas, conclui-se que as tabelas apresentam alta qualidade estrutural.

**Chaves primárias sem duplicatas:**
- `transaction_id`: 52.000 únicos
- `customer_id`: 2.500 únicos
- `merchant_id`: 1.000 únicos

**Identificadores secundários consistentes:**
- `cpf_cnpj`, `cnpj`, `full_name`, `merchant_name`: sem duplicatas
- `device_fingerprint`, `ip_address`: únicos por transação

**Conclusão**: Não há necessidade de tratamento de duplicatas.

In [21]:
unique_values(df_transactions, id_columns=['transaction_id'])

transaction_id: 0 duplicates


,Unique Values,Total Values,Duplicates
transaction_id,52000,52000,0
timestamp,51837,52000,163
transaction_type,3,52000,51997
sender_id,3497,52000,48503
sender_entity_type,2,52000,51998
receiver_id,3485,52000,48515
receiver_entity_type,2,52000,51998
amount_brl,50201,52000,1799
amount_orig,50191,52000,1809
currency,3,52000,51997


In [22]:
unique_values(df_kyc_profiles, id_columns=['customer_id'])

customer_id: 0 duplicates


,Unique Values,Total Values,Duplicates
customer_id,2500,2500,0
full_name,2500,2500,0
cpf_cnpj,2500,2500,0
date_of_birth,2388,2500,112
annual_income_brl,2397,2500,103
declared_occupation,16,2500,2484
risk_rating,3,2500,2497
registration_date,1461,2500,1039
country,1,2500,2499
state,14,2500,2486


In [23]:
unique_values(df_merchants, id_columns=['merchant_id'])

merchant_id: 0 duplicates


,Unique Values,Total Values,Duplicates
merchant_id,1000,1000,0
merchant_name,1000,1000,0
cnpj,1000,1000,0
mcc,14,1000,986
country,20,1000,980
owner_customer_id,319,349,30
merchant_high_risk_flag,2,1000,998
merchant_chargeback_ratio_90d,180,1000,820
mcc_risk,2,1000,998


### **6. Relacionamento entre os dataframes**

As tabelas se conectam via `customer_id`, `merchant_id`, `sender_id` e `receiver_id`  viabilizando análises multidimensionais.

In [24]:
# Verificar se todas as transações possuem cliente ou comerciante associado em sender_id
sender_ids = set(df_transactions['sender_id'].dropna().unique())
customer_ids = set(df_kyc_profiles['customer_id'].dropna().unique())
merchant_ids = set(df_merchants['merchant_id'].dropna().unique())
associated_ids = customer_ids.union(merchant_ids)

unassociated_senders = sender_ids - associated_ids
sender_as_customers = sender_ids.intersection(customer_ids)
sender_as_merchants = sender_ids.intersection(merchant_ids)

if unassociated_senders:
    print(f"\nExemplos de sender_ids não associados (até 20):")
    for sid in sorted(list(unassociated_senders))[:20]:
        count = (df_transactions['sender_id'] == sid).sum()
        print(f" - {sid}: {count} transações")

summary_senders = pd.DataFrame({
    'Categoria': ['Clientes', 'Comerciantes', 'Ambos (clientes e comerciantes)', 'Não Associados'],
    'Quantidade': [
        len(sender_as_customers - sender_as_merchants),
        len(sender_as_merchants - sender_as_customers),
        len(sender_as_customers & sender_as_merchants),
        len(unassociated_senders)
    ]
})

display(summary_senders)

,Categoria,Quantidade
0,Clientes,2500
1,Comerciantes,997
2,Ambos (clientes e comerciantes),0
3,Não Associados,0


In [25]:
# Verificar se todas as transações possuem cliente ou comerciante associado em receiver_id
receiver_ids = set(df_transactions['receiver_id'].dropna().unique())
customer_ids = set(df_kyc_profiles['customer_id'].dropna().unique())
merchant_ids = set(df_merchants['merchant_id'].dropna().unique())
associated_ids = customer_ids.union(merchant_ids)

unassociated_receivers = receiver_ids - associated_ids
receiver_as_customers = receiver_ids.intersection(customer_ids)
receiver_as_merchants = receiver_ids.intersection(merchant_ids)

if unassociated_receivers:
    print(f"\nExemplos de receiver_ids não associados (até 20):")
    for rid in sorted(list(unassociated_receivers))[:20]:
        count = (df_transactions['receiver_id'] == rid).sum()
        print(f" - {rid}: {count} transações")

summary_receivers = pd.DataFrame({
    'Categoria': ['Clientes', 'Comerciantes', 'Ambos (clientes e comerciantes)', 'Não Associados'],
    'Quantidade': [
        len(receiver_as_customers - receiver_as_merchants),
        len(receiver_as_merchants - receiver_as_customers),
        len(receiver_as_customers & receiver_as_merchants),
        len(unassociated_receivers)
    ]
})

display(summary_receivers)


,Categoria,Quantidade
0,Clientes,2485
1,Comerciantes,1000
2,Ambos (clientes e comerciantes),0
3,Não Associados,0


### **Conclusão**
Após análise das quatro bases de dados fornecidas (transações, KYC, merchants e geo behavior), conclui-se que:

**Integridade Estrutural**: O dataset apresenta 52.000 transações com identificadores únicos, 2.500 clientes e 1.000 merchants, todos com chaves primárias íntegras e relacionamentos válidos entre tabelas.

**Completude dos Dados**: Os valores ausentes identificados seguem padrões estruturais e não representam problemas de qualidade

**Consistência**: Tipos de dados apropriados e padronizados.

**Cobertura para análise AML**: O dataset contém todos os elementos necessários para análise aprofundada de risco: identificadores de clientes e merchants, indicadores PEP, risk scores, sinais de risco comportamentais e operacionais, informações geográficas e características transacionais detalhadas.

Os dados encontram-se íntegros, completos e prontos para a etapa de análise, sem necessidade de tratamentos adicionais.

In [26]:
# Salvar dataframes em formato Parquet
df_transactions.to_parquet('data/processed/transactions.parquet', index=False, engine='fastparquet', compression='snappy')
df_kyc_profiles.to_parquet('data/processed/kyc_profiles.parquet', index=False, engine='fastparquet', compression='snappy')
df_merchants.to_parquet('data/processed/merchants.parquet', index=False, engine='fastparquet', compression='snappy')
df_geo_behavior.to_parquet('data/processed/geo_behavior.parquet', index=False, engine='fastparquet', compression='snappy')